In [17]:
from autograd import make_jvp, jacobian
import autograd.numpy as np

**Objective**: Test jacobian calculation in python

First define the function

Autograd does not support assignment to index, so construct $\frac{dy}{dt}$ using stack instead of indexing

In [86]:
def f(x1,x2,x3,x4,x5,x6):
    # State is { x,y,theta, vx,vy,omega }
    dydt = []
    dydt.append( x4**2 + 2*x6 )
    dydt.append( 5*x5 )
    dydt.append( x4 + 2*x5 + 3*x6 )
    dydt.append( x1**2 + 2*x2 )
    dydt.append( x2**3 + 4*x3 )
    dydt.append( 0 )
    
    return np.stack(dydt)

Test that the calculation is what we expect

In [102]:
x = np.ones(6)
res = []
for i in range(6):
    res.append( jacobian(f, argnum=i)(*x) )
res = np.stack(res, axis=1)
print(res)

[[0. 0. 0. 2. 0. 2.]
 [0. 0. 0. 0. 5. 0.]
 [0. 0. 0. 1. 2. 3.]
 [2. 2. 0. 0. 0. 0.]
 [0. 3. 4. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


## Compare speed of `Jacobian` vs `make_jvp`

make_jvp is expected to be much faster because it is a fast forward-mode Jacobian

### `Jacobian`

In [99]:
%%timeit
x = np.zeros(6)
J = [ jacobian(f, argnum=i) for i in range(6) ]
for t in range(1000):
    res = []
    for i in range(6):
        res.append( J[i](*x) )
    res = np.stack(res).T

3.07 s ± 17.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### `make_jvp`

In [100]:
%%timeit
x = np.zeros(6)
J = [ make_jvp(f, argnum=i) for i in range(6) ]
for t in range(1000):
    res = []
    for i in range(6):
        res.append( J[i](*x) )
    res = np.stack(res).T

45.3 ms ± 316 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### Conclusion
`make_ivp` is around 70x faster than `jacobian`